In [1]:
import os
import subprocess
from concurrent.futures import ThreadPoolExecutor
# 데이터셋, 시드, 알파 값, 그리고 GPU 장치 번호 설정
datasets = ['freesolv','esol','lipo','qm7','qm8', 'bbbp', 'bace', 'sider', 'toxcast', 'tox21', 'clintox', 'hiv']
seeds = [777, 778, 789]
devices = [0, 1]  # 사용 가능한 GPU 장치 번호

# 실행할 스크립트
script = 'pretrain2+finetune-fragment.py'

def run_script(script, dataset, seed, device):
    # pretrained model location
    input_model_directory = f"./pretrain2/fragment/zinc_standard_agent_gin_0.25_42/"

    input_model_file = os.path.join(input_model_directory, 'model.pth')

    # 기본 커맨드 설정
    command = [
        'python', script,
        '--dataset', dataset,
        '--device', str(device),
        '--seed', str(seed),
        '--input_model_file', input_model_file
    ]

    # 현재 디렉토리 변경
    os.chdir('/tf/GraphMAE-pyg - copy/chem')

    # subprocess를 사용하여 명령어 실행실행
    subprocess.run(command, check=True)


# ThreadPoolExecutor로 병렬 실행
with ThreadPoolExecutor(max_workers=20) as executor:
    futures = []
    for i, (dataset, seed ) in enumerate([(d, s) for d in datasets for s in seeds ]):
        device = devices[i % len(devices)]  # GPU 장치 번호 순환
        # 각 조합에 대해 스크립트 실행을 스케줄
        future = executor.submit(run_script, script, dataset,seed, device)
        futures.append(future)
    
    # 결과 대기
    for future in futures:
        future.result()  # 실행 결과 확인


[00:35:38] WARNING: not removing hydrogen atom without neighbors
[00:35:38] WARNING: not removing hydrogen atom without neighbors
[00:35:38] WARNING: not removing hydrogen atom without neighbors
[00:35:39] WARNING: not removing hydrogen atom without neighbors
[00:35:39] WARNING: not removing hydrogen atom without neighbors
[00:35:39] WARNING: not removing hydrogen atom without neighbors
[00:35:39] WARNING: not removing hydrogen atom without neighbors
[00:35:39] WARNING: not removing hydrogen atom without neighbors
[00:35:39] WARNING: not removing hydrogen atom without neighbors
[00:35:39] WARNING: not removing hydrogen atom without neighbors
[00:35:39] WARNING: not removing hydrogen atom without neighbors
[00:35:39] WARNING: not removing hydrogen atom without neighbors
[00:35:39] WARNING: not removing hydrogen atom without neighbors
[00:35:39] WARNING: not removing hydrogen atom without neighbors
[00:35:39] WARNING: not removing hydrogen atom without neighbors
[00:35:39] WARNING: not r

MoleculeDataset(6834) qm7 777
scaffold


[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not r

MoleculeDataset(6834) qm7 778
scaffold
MoleculeDataset(6834) qm7 789
scaffold


[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:40] WARNING: not removing hydrogen atom without neighbors
[00:35:41] [00:35:41] WARNING: not removing hydrogen atom without neighbors
[00:35:41] WARNING: not removing hydrogen atom without neighbors
[00:35:41] WARNING: not removing hydrogen atom without neighbors
[00:35:41] WARNING: not removing hydrogen atom without neighbors
[00:35:41] WARNING: not removing hydrogen atom without neighbors
[00:35:41] WAR

...

train_loss : inf val_loss: inf
train: 0.000000 val: 0.814622  best_val_acc: 0.814622
Loaded trained model with success.


Iteration:  89%|████████▉ | 914/1028 [00:13<00:01, 71.90it/s]

test auc, test_loss : 0.7474883640085748 tensor(0.0844, device='cuda:1', dtype=torch.float64)
Extracted ratio: 0.25


Iteration: 100%|██████████| 1028/1028 [00:15<00:00, 67.77it/s]


====Evaluation
omit the training accuracy computation
train_loss : inf val_loss: inf
train: 0.000000 val: 0.823269  best_val_acc: 0.823269
Loaded trained model with success.
test auc, test_loss : 0.7607640163000444 tensor(0.0920, device='cuda:1', dtype=torch.float64)
Extracted ratio: 0.25
